In [2]:
# Code to be able to import local modules in notebooks
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
### Import dependencies
# Standard python libraries
import os
# Third party libraries
import pandas as pd
# Custom packages
from ffpackage.scraping import mfl
from ffpackage.scraping import ourlads
from ffpackage.analysis import analysis
from ffpackage.viz import viz
from appmanager.database import db

In [4]:
# Find environment variables
DATABASE_URL = os.environ.get("DATABASE_URL", None)
# sqlalchemy deprecated urls which begin with "postgres://"; now it needs to start with "postgresql://"
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)


In [5]:
# Retrieve basic info on all players
allPlayers = mfl.get_players()
# Retrieve Fantasy Sharks rankings
sharkRanks = mfl.get_sharkRanks()
# Retrieve Average Draft Positions
adpValues = mfl.get_adp()

# Find player ages
# Retrieve any player dobs who are already in the db
player_dobs = db.get_df('player_dobs')
# Rename the dob columns until I can get to that in the app setup
player_dobs = player_dobs.rename(columns={'PlayerID':'id_mfl', 'DOB':'dob', 'Age':'age'})
# Check for any players whose ages are not already in the db
ageNeeded = allPlayers.loc[~allPlayers['id_mfl'].isin(player_dobs['id_mfl'])]
# If there are players whose ages are not in the db, retrieve those ages from My Fantasy League
if len(ageNeeded)>0:
    # Break player list into chunks small enough for the API server
    # Specify chunk size
    n = 50
    # Slice the df where ages are needed into chunks
    list_df = [ageNeeded['id_mfl'][i:i+n] for i in range(0,ageNeeded['id_mfl'].shape[0],n)]
    # Loop over the chunks
    for i in range(len(list_df)):
        # Join the playerIDs into a string to use in building the API call
        idList = ",".join(list_df[i])
        # Retrieve the dates of birth for these players from My Fantasy League
        new_dobs = mfl.get_playerProfiles(idList)
        # Append these players to the list that already existed in the app's database
        player_dobs = player_dobs.append(new_dobs)
# Convert date columns to datetime
player_dobs['dob'] = pd.to_datetime(player_dobs['dob'])
# Convert Date of Birth to Age using datetime's relativedelta module
player_dobs['age'] = player_dobs['dob'].apply(analysis.calculate_age)
# Drop the dob column
player_dobs = player_dobs.drop(columns='dob')

# Merge all dfs from MyFantasyLeague API
allPlayers = allPlayers.merge(
    player_dobs, on='id_mfl', how='left'
).merge(
    sharkRanks, on='id_mfl', how='left'
).merge(
    adpValues, on='id_mfl', how='left'
)
# Clean sharkRank and ADP columns
# If a player is ranked low enough, they do not get a ranking in FantasySharks or ADP. Replace these with a very low rank
allPlayers['sharkRank'].fillna(3000, inplace=True)
allPlayers['adp'].fillna(3000, inplace=True)

/Users/travisharry/Desktop/Coding/ff_demo2/ffpackage/scraping/mfl.py:36: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'playerName'] = df.loc[:, 'playerName'].str.replace(".", "")
/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)
/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_5287/4278180581.py:21: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get th

In [6]:
# Retrieve position ranks from OurLads
posRanks = ourlads.scrape_depthcharts()

/Users/travisharry/Desktop/Coding/ff_demo2/ffpackage/scraping/ourlads.py:78: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'playerName'] = df.loc[:, 'playerName'].str.replace(".", "")


In [7]:
### Merge MyFantasyLeague data with OurLads data
allPlayers = allPlayers.merge(posRanks, how='left', on=['playerName', 'team'])
## Clean merged df
# Give defenses a position rank since OurLads does not include them
allPlayers.loc[allPlayers['pos']=='DF', 'posRank'] = "DF1"
# Give any players who were not on the OurLads website a rank of 3
allPlayers.loc[allPlayers['posRank'].isna(), 'posRank'] = allPlayers.loc[allPlayers['posRank'].isna(), 'pos'] + "3"
# Give any injured reserve players a rank of 3
allPlayers.loc[allPlayers['posRank']=="RES", 'posRank'] = allPlayers.loc[allPlayers['posRank']=="RES", 'pos'] + "3"
# Specify all players are in current season
allPlayers['season'] = 2022

In [9]:
### Fetch historical data from app database
prior1 = db.get_df('prior1')
prior2 = db.get_df('prior2')

/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)
/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


In [8]:
allPlayers

,id_mfl,playerName,pos,team,age,sharkRank,adp,posRank,PR,KR,RES,season
0,0501,BUFFALO BILLS,DF,BUF,NaN,225.0,3000.0,DF1,NaN,NaN,NaN,2022
1,0502,INDIANAPOLIS COLTS,DF,IND,NaN,800.0,3000.0,DF1,NaN,NaN,NaN,2022
2,0503,MIAMI DOLPHINS,DF,MIA,NaN,584.0,3000.0,DF1,NaN,NaN,NaN,2022
3,0504,NEW ENGLAND PATRIOTS,DF,NEP,NaN,410.0,3000.0,DF1,NaN,NaN,NaN,2022
4,0505,NEW YORK JETS,DF,NYJ,NaN,699.0,3000.0,DF1,NaN,NaN,NaN,2022
...,...,...,...,...,...,...,...,...,...,...,...,...
1111,16080,RASHID SHAHEED,WR,NOS,24.0,808.0,3000.0,WR2,YES,YES,NaN,2022
1112,16093,DRE MILLER,TE,NYG,24.0,1188.0,3000.0,TE3,NaN,NaN,NaN,2022
1113,16095,NICK SCIBA,PK,PIT,23.0,3000.0,3000.0,PK3,NaN,NaN,NaN,2022
1114,16096,STANLEY BERRYHILL,WR,DET,24.0,1044.0,3000.0,WR2,NaN,NaN,NaN,2022
